<a href="https://colab.research.google.com/github/quanap5/ColabNote2/blob/master/Custom_Gradient_Descent_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

## Thực hiện Gradient Decent với thư viện Tensorflow không sử dụng Built-in optimizer

Một điểm thú vị trên thư viện Tensorflow đó là nó có khả năng tự động tính gradient của một hàm bất kỳ. Tất cả những gì chúng ta phải làm chỉ là setup function đó và chạy lệnh tf.gradients để tính toán gradient của hàm đó. Chúng ta không cần quan tâm quá trình tính toán gồm những bước nào từ việc tính toán lossfunction. Và hôm nay chúng ta sẽ tìm hiểu về vấn đề này.

Hãy nhìn vào vài ví dụ sau để thấy rõ được vấn đề.



### 1.1 Sử dụng Softmax Regression trên MNIST dataset sử dụng Optimizer tích hợp sẵn trong tensorflow

Trước khi đi sâu vào tìm hiểu tf.Gradients, chúng ta sẽ bắt đầu với ví dụ giair bài toán MNIST sử dụng Optimizer có sẵn trong thư viện Tensorflow (Gradient descent optimization algorithm)

In [2]:
import tensorflow as tf

# MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

# Parameter
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

# tf graph input

x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28 = 784
y = tf.placeholder(tf.float32, [None, 10])  # 0-9 digits recognition => 10 classes

# set model weight
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# construct model
pred = tf.nn.softmax(tf.matmul(x, W)+ b) # softmax

# minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Start training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
#             print(__w)
            
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
#             print(sess.run(W))
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

W0717 03:00:12.892016 140114936362880 deprecation.py:323] From <ipython-input-2-8e298ed8ff5a>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0717 03:00:12.893603 140114936362880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0717 03:00:12.894768 140114936362880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0717 03:00:13.236477 140114936362880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0717 03:00:13.239918 140114936362880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0717 03:00:13.316047 140114936362880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost= 1.183742214
Epoch: 0002 cost= 0.665294645
Epoch: 0003 cost= 0.552712628
Epoch: 0004 cost= 0.498699845
Epoch: 0005 cost= 0.465540914
Epoch: 0006 cost= 0.442628153
Epoch: 0007 cost= 0.425437523
Epoch: 0008 cost= 0.412244748
Epoch: 0009 cost= 0.401447279
Epoch: 0010 cost= 0.392452632
Optimization Finished!
Accuracy: 0.87266666


### 1. 2 Softmax Regression trên data MNIST using tf.gradient

Để thực hiện Gradient Descent chúng ta sẽ bỏ phần code liên quan đến Optimizer ở trên. Thay vào đó chúng ta sẽ tự viết phần update weight cho netwwork như sau.

In [3]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 10
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

grad_W, grad_b = tf.gradients(xs=[W, b], ys=cost)


new_W = W.assign(W - learning_rate * grad_W)
new_b = b.assign(b - learning_rate * grad_b)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, _,  c = sess.run([new_W, new_b ,cost], feed_dict={x: batch_xs,
                                                      y: batch_ys})
            
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
#             print(sess.run(W))
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost= 0.549140360
Epoch: 0002 cost= 0.366004887
Epoch: 0003 cost= 0.336440942
Epoch: 0004 cost= 0.321183545
Epoch: 0005 cost= 0.311251386
Epoch: 0006 cost= 0.304264489
Epoch: 0007 cost= 0.298515269
Epoch: 0008 cost= 0.294186639
Epoch: 0009 cost= 0.290741280
Epoch: 0010 cost= 0.287484975
Epoch: 0011 cost= 0.284897734
Epoch: 0012 cost= 0.282543033
Epoch: 0013 cost= 0.280638179
Epoch: 0014 cost= 0.278797283
Epoch: 0015 cost= 0.277104856
Epoch: 0016 cost= 0.275465670
Epoch: 0017 cost= 0.274080394
Epoch: 0018 cost= 0.272911114
Epoch: 0019 cost= 0.271534524
Epoch: 0020 cost= 0.270761878
Epoch: 0021 cost= 0.269505150
Epoch: 0022 cost= 0.268630186
Epoch: 0023 cost= 0.267730661
Epoch: 0024 cost= 0.266859166
Epoch: 0025 cost= 0.266102107
Epoch: 0026 cost= 0.265243174
Epoch: 0027 cost= 0.264610580
